In [ ]:
import os

os.chdir("..")

current_directory = os.getcwd()
print("Current Working Directory:", current_directory)


In [ ]:
# Import necessary modules and functions
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from src.neural_nets import create_network_1, create_network_2, create_network_3, parameter_tuning_nn, runs_nn
from src.knn_regression import parameter_tuning_knn, runs_knn, generate_neighbors
from src.tree_regression import parameter_tuning_tree, runs_tree
from src.rbf_interpolation import runs_rbf
from data.data_generator import get_data, preprocess
from data.regression_functions import m1, m2, m3, m4, m5, m6, m7, m8

In [3]:
# Define a list of regression functions
regression_functions = [m1, m2, m3, m4, m5, m6, m7, m8]

## Neural Networks Regression

In [ ]:
# Define a list of neural networks
neural_networks = [create_network_1, create_network_2, create_network_3]


# Define empty dataframe to store results
results_nn = pd.DataFrame(
    columns=["regression_function", "network", "units", "mses", "maes"]
)

median_and_iqr_nn = pd.DataFrame(
    columns=["regression_function", "network", "units", "mse", "iqr"]
)

# Loop through each regression function
for i, regression_func in enumerate(regression_functions):
    print(f"Regression Function {i+1}: {regression_func.__name__}")
    input_dim = regression_func.expected_dim
    x, y = get_data(regression_func, x_dim=input_dim, num_samples=1000, sigma=0.05)
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

    # Preprocess data
    train_data = preprocess(x_train, y_train, training=True)
    test_data = preprocess(x_test, y_test, training=False)

    # Tune networks
    units = [3, 6, 9, 12, 15]
    for j, network in enumerate(neural_networks):
        best_params = parameter_tuning_nn(
            network,
            units=units,
            train_data=train_data,
            test_data=test_data,
            input_dim=input_dim,
        )

        # Calculate regression for best units
        best_units = best_params["best_config"]
        mses, maes = runs_nn(
            network,
            units=best_units,
            input_dim=input_dim,
            regression_func=regression_func,
        )

        # Calculate median and IQR
        mse = np.median(mses)
        iqr = np.subtract(*np.percentile(mses, [75, 25]))

        # Append the results to the dataframe
        df_nn = pd.DataFrame(
            {
                "regression_function": [regression_func.__name__],
                "network": [network.__name__],
                "units": [best_units],
                "mse": [mse],
                "iqr": [iqr],
            }
        )

        mses_maes_nn = pd.DataFrame(
            {
                "regression_function": [regression_func.__name__],
                "network": [network.__name__],
                "units": [best_units],
                "mses": [mses],
                "maes": [maes],
            }
        )
        
        results_nn = pd.concat([results_nn, mses_maes_nn], ignore_index=True)
        median_and_iqr_nn = pd.concat([median_and_iqr_nn, df_nn], ignore_index=True)

        print(f"Best Units: {best_units}")
        print(f"MSE: {mse}")
        print(f"IQR: {iqr}")

    print("--------------------")

In [ ]:
# Define the file path
file_path = "results/nn_performance.csv"

# Save the results to a csv file
results_nn.to_csv(file_path, index=False)

## KNN-Regression

In [ ]:
# Define empty dataframe to store results
results_knn = pd.DataFrame(columns=["regression_function", "units", "mses", "maes"])
median_and_iqr_knn = pd.DataFrame(
    columns=["regression_function", "units", "mse", "iqr"]
)

# Loop through each regression function
for i, regression_func in enumerate(regression_functions):
    print(f"Regression Function {i+1}: {regression_func.__name__}")
    input_dim = regression_func.expected_dim
    x, y = get_data(regression_func, x_dim=input_dim, num_samples=1000, sigma=0.05)
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

    # Preprocess data
    train_data = (x_train, y_train)
    test_data = (x_test, y_test)

    # Tune number of neighbors
    units = generate_neighbors(x_train.shape[0], 25)
    best_params = parameter_tuning_knn(
        units=units,
        train_data=train_data,
        test_data=test_data,
    )

    # Calculate regression for the best number of neighbors
    best_units = best_params["best_config"]
    mses, maes = runs_knn(
        unit=best_units,
        input_dim=input_dim,
        regression_func=regression_func,
    )

    mse = np.median(mses)
    iqr = np.subtract(*np.percentile(mses, [75, 25]))

    # Append the results to the dataframe
    df_knn = pd.DataFrame(
        {
            "regression_function": [regression_func.__name__],
            "units": [best_units],
            "mse": [mse],
            "iqr": [iqr],
        }
    )

    mses_maes_knn = pd.DataFrame(
        {
            "regression_function": [regression_func.__name__],
            "units": [best_units],
            "mses": [mses],
            "maes": [maes],
        }
    )

    median_and_iqr_knn = pd.concat([median_and_iqr_knn, df_knn], ignore_index=True)
    results_knn = pd.concat([results_knn, mses_maes_knn], ignore_index=True)

    print(f"Best Units: {best_units}")
    print(f"MSE: {mse}")
    print(f"IQR: {iqr}")

    print("--------------------")

In [6]:
# Define the file path
file_path = "results/knn_performance.csv"

# Save the results to a csv file
results_knn.to_csv(file_path, index=False)

## Tree Regression

In [ ]:
# Define empty dataframe to store results
results_tree = pd.DataFrame(columns=["regression_function", "max_depth", "max_leaf_nodes", "mses", "maes"])
median_and_iqr_tree = pd.DataFrame(
    columns=["regression_function", "max_depth", "max_leaf_nodes", "mse", "iqr"]
)

# Loop through each regression function
for i, regression_func in enumerate(regression_functions):
    print(f"Regression Function {i+1}: {regression_func.__name__}")
    input_dim = regression_func.expected_dim
    x_train, y_train = get_data(regression_func, x_dim=input_dim, num_samples=800, sigma=0.05)

    # Preprocess data
    train_data = (x_train, y_train)

    # Tune number of leafs an depth
    best_params = parameter_tuning_tree(
        train_data=train_data,
    )

    max_depth = best_params["max_depth"]
    max_leaf_nodes = best_params["max_leaf_nodes"]

    # Calculate regression for the best number of neighbors
    mses, maes = runs_tree(
        max_depth=max_depth,
        max_leaf_nodes=max_leaf_nodes,
        input_dim=input_dim,
        regression_func=regression_func,
    )

    mse = np.median(mses)
    iqr = np.subtract(*np.percentile(mses, [75, 25]))

    # Append the results to the dataframe
    df_tree = pd.DataFrame(
        {
            "regression_function": [regression_func.__name__],
            "max_depth": max_depth,
            "max_leaf_nodes": max_leaf_nodes,
            "mse": [mse],
            "iqr": [iqr],
        }
    )

    mses_maes_tree = pd.DataFrame(
        {
            "regression_function": [regression_func.__name__],
            "max_depth": max_depth,
            "max_leaf_nodes": max_leaf_nodes,
            "mses": [mses],
            "maes": [maes],
        }
    )

    median_and_iqr_tree = pd.concat([median_and_iqr_tree, df_tree], ignore_index=True)
    results_tree = pd.concat([results_tree, mses_maes_tree], ignore_index=True)

    print(f"max_depth: {max_depth}")
    print(f"max_leaf_nodes: {max_leaf_nodes}")
    print(f"MSE: {mse}")
    print(f"IQR: {iqr}")

    print("--------------------")

## RBF-Interpolation

In [ ]:
# Define empty dataframe to store results
results_rbf = pd.DataFrame(columns=["regression_function", "mses", "maes"])
median_and_iqr_rbf = pd.DataFrame(columns=["regression_function", "mse", "iqr"])

# Loop through each regression function
for i, regression_func in enumerate(regression_functions):
    print(f"Regression Function {i+1}: {regression_func.__name__}")
    input_dim = regression_func.expected_dim
    
    # Calculate regression for the best number of neighbors
    mses, maes = runs_rbf(
        input_dim=input_dim,
        regression_func=regression_func,
    )

    mse = np.median(mses)
    iqr = np.subtract(*np.percentile(mses, [75, 25]))

    # Append the results to the dataframe
    df_rbf = pd.DataFrame(
        {
            "regression_function": [regression_func.__name__],
            "mse": [mse],
            "iqr": [iqr],
        }
    )

    mses_maes_rbf = pd.DataFrame(
        {
            "regression_function": [regression_func.__name__],
            "mses": [mses],
            "maes": [maes],
        }
    )

    median_and_iqr_rbf = pd.concat([median_and_iqr_rbf, df_rbf], ignore_index=True)
    results_rbf = pd.concat([results_rbf, mses_maes_rbf], ignore_index=True)

    print(f"MSE: {mse}")
    print(f"IQR: {iqr}")

    print("--------------------")